In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from scipy.stats import ttest_1samp

In [16]:
processed_dataset = pd.read_csv('data/processed-data.csv')
processed_dataset.head()

,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,y_labels
0,0.284211,0.340000,0.056604,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.00,0.000000,0.0,0.372093,0.488889,0.766182,Dropout,-1
1,0.684211,0.500000,0.037736,0.0,0.230769,0.133333,0.230769,0.741722,0.0,0.0,0.26087,0.181818,0.30,0.735897,0.0,0.732558,0.111111,0.640687,Graduate,1
2,0.284211,0.313684,0.037736,0.0,0.230769,0.000000,0.000000,0.000000,0.0,0.0,0.26087,0.000000,0.00,0.000000,0.0,0.372093,0.488889,0.766182,Dropout,-1
3,0.284211,0.258947,0.056604,0.0,0.230769,0.177778,0.230769,0.711447,0.0,0.0,0.26087,0.303030,0.25,0.667692,0.0,0.209302,0.000000,0.124174,Graduate,1
4,0.052632,0.489474,0.528302,0.0,0.230769,0.200000,0.192308,0.653422,0.0,0.0,0.26087,0.181818,0.30,0.700000,0.0,0.732558,0.111111,0.640687,Graduate,1


In [17]:
feature_columns = processed_dataset.columns.drop(['Target', 'y_labels'])
X = processed_dataset[feature_columns].to_numpy(dtype=float)
y = processed_dataset['y_labels'].to_numpy(dtype=int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [19]:
# Augment X matrices with column of 1s (for intercept)
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

In [20]:
# Train a soft-margin linear SVM via SGD using hinge loss and regularization term C
def train_linear_svm(X, y, C=0.01, T=1000):
    """
    Runs T iterations of SGD on the hinge loss SVM objective.
    Returns the averaged weight vector w_bar.
    """
    n, d = X.shape
    theta = np.zeros(d)
    w_sum = np.zeros(d)

    for t in range(1, T + 1):
        # form the current predictor
        w_t = theta / (2 * C * t)

        # uniformly pick one point at random 
        i = np.random.randint(n)

        if y[i] * (w_t.dot(X[i])) < 1:
            # only on violations do we update theta
            theta = theta + y[i] * X[i]
            
        # otherwise theta stays the same

        w_sum += w_t

    # return the average of all w_t’s
    return w_sum / T


In [21]:
# Testing model on testing set
w_final = train_linear_svm(X_train, y_train, T=1000)

In [22]:
coeffs    = w_final[:-1]
intercept = w_final[-1]

print(f"Intercept: {intercept:.4f}\n")
print("Feature Coefficients:")
for name, coeff in zip(feature_columns, coeffs):
    print(f"- {name:50s} {coeff: .4f}")

Intercept: -0.5773

Feature Coefficients:
- Previous qualification (grade)                     -0.0590
- Admission grade                                    -0.0077
- Age at enrollment                                  -0.5452
- Curricular units 1st sem (credited)                -0.0174
- Curricular units 1st sem (enrolled)                -0.2587
- Curricular units 1st sem (evaluations)             -0.2724
- Curricular units 1st sem (approved)                 0.5804
- Curricular units 1st sem (grade)                    0.8073
- Curricular units 1st sem (without evaluations)     -0.0482
- Curricular units 2nd sem (credited)                 0.0207
- Curricular units 2nd sem (enrolled)                -0.3253
- Curricular units 2nd sem (evaluations)             -0.0777
- Curricular units 2nd sem (approved)                 1.0302
- Curricular units 2nd sem (grade)                    1.9480
- Curricular units 2nd sem (without evaluations)     -0.1016
- Unemployment rate                        

In [23]:
# Test Accuracy
test_preds = np.sign(X_test.dot(w_final))
test_acc = (test_preds == y_test).mean()
print(f"\nTest Accuracy: {test_acc:.3f}")


Test Accuracy: 0.820


For further evaluation, we'll create a dual-optimization (kernel) SVM to compare. To ensure the feature set of this dual-SGD SVM is robust, we’ll first run five-fold cross-validation on this linear SVM using fixed folds each time in order to evaluate each coefficient via statistical analysis. Features that fail to reach some threshold (i.e. features with p-value >= 0.05) will be removed. We expect that this will improve the performance of the subsequent kernel-based SVM.

In [24]:
kf = KFold(n_splits=5, shuffle=False) # shuffle=False ensures folds remain fixed across iterations
coefs, val_accs = [], []

In [25]:
for tr_idx, val_idx in kf.split(X_train):
    X_tr, y_tr = X_train[tr_idx], y_train[tr_idx]
    X_val, y_val = X_train[val_idx], y_train[val_idx]
    
    w_fold = train_linear_svm(
        X_tr, 
        y_tr,
    )
    coefs.append(w_fold)
    
    preds = np.sign(X_val.dot(w_fold))
    val_accs.append((preds == y_val).mean())

coefs = np.vstack(coefs)

In [26]:
print("Validation Accuracies:", val_accs)
print(f"Mean Cross-Validation Accuracy: {np.mean(val_accs):.3f} ± {np.std(val_accs, ddof=1):.3f}")

Validation Accuracies: [0.8008474576271186, 0.8107344632768362, 0.806497175141243, 0.8036723163841808, 0.8033946251768034]
Mean Cross-Validation Accuracy: 0.805 ± 0.004


In [27]:
# Statistical Analysis on 5-fold Cross Validation
coef_mean = coefs.mean(axis=0)
coef_std  = coefs.std(axis=0, ddof=1)
t_stats, p_vals = ttest_1samp(coefs, popmean=0.0, axis=0)

results = pd.DataFrame({
    'feature': list(feature_columns) + ['bias'],
    'mean_w': coef_mean,
    'std_w' : coef_std,
    't_stat': t_stats,
    'p_value': p_vals
}).sort_values('p_value')

results

,feature,mean_w,std_w,t_stat,p_value
12,Curricular units 2nd sem (approved),1.187810,0.103578,25.642668,0.000014
18,bias,-0.597130,0.061461,-21.724668,0.000027
13,Curricular units 2nd sem (grade),1.856012,0.191950,21.621134,0.000027
2,Age at enrollment,-0.787903,0.120257,-14.650351,0.000126
6,Curricular units 1st sem (approved),0.671046,0.119692,12.536371,0.000233
7,Curricular units 1st sem (grade),0.921117,0.208184,9.893572,0.000586
15,Unemployment rate,-0.195260,0.055855,-7.816890,0.001446
10,Curricular units 2nd sem (enrolled),-0.160337,0.077423,-4.630743,0.009802
4,Curricular units 1st sem (enrolled),-0.129548,0.071758,-4.036904,0.015644
1,Admission grade,0.158843,0.115114,3.085490,0.036735


All features with a p-value less than 0.05 are statistically significant and will be kept. Features with higher p-values will be excluded from the model.

Significant Features (p < 0.05):
- Curricular units 2nd sem (approved)
- Curricular units 2nd sem (grade)
- Age at enrollment
- Curricular units 1st sem (approved)
- Curricular units 1st sem (grade)
- Unemployment rate
- Curricular units 2nd sem (enrolled)
- Curricular units 1st sem (enrolled)
- Admission grade
- Inflation rate

Non-Significant Features (p ≥ 0.05):
- Curricular units 1st sem (evaluations)
- Curricular units 2nd sem (without evaluations)
- Curricular units 1st sem (without evaluations)
- Curricular units 2nd sem (evaluations)
- Curricular units 1st sem (credited)
- Curricular units 2nd sem (credited)
- GDP
- Previous qualification (grade)

In [28]:
# Removing non-significant features (p >= 0.05)
nonsignificant_features = [
    'Curricular units 1st sem (evaluations)',
    'Curricular units 2nd sem (without evaluations)',
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 1st sem (credited)',
    'Curricular units 2nd sem (credited)',
    'GDP',
    'Previous qualification (grade)'
]

processed_dataset = processed_dataset.drop(columns=nonsignificant_features, errors='ignore', axis=1)
processed_dataset.head()

,Admission grade,Age at enrollment,Curricular units 1st sem (enrolled),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Unemployment rate,Inflation rate,Target,y_labels
0,0.340000,0.056604,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.372093,0.488889,Dropout,-1
1,0.500000,0.037736,0.230769,0.230769,0.741722,0.26087,0.30,0.735897,0.732558,0.111111,Graduate,1
2,0.313684,0.037736,0.230769,0.000000,0.000000,0.26087,0.00,0.000000,0.372093,0.488889,Dropout,-1
3,0.258947,0.056604,0.230769,0.230769,0.711447,0.26087,0.25,0.667692,0.209302,0.000000,Graduate,1
4,0.489474,0.528302,0.230769,0.192308,0.653422,0.26087,0.30,0.700000,0.732558,0.111111,Graduate,1


In [29]:
processed_dataset.to_csv('data/significant-features-data.csv', index=False)